In [1]:
import torch
from torchvision import models, transforms
from PIL import Image
import io
from scipy.spatial.distance import cosine
from transformers import AutoModel
import numpy as np
from transformers import AutoModel
from skimage.metrics import structural_similarity as ssim

c:\Users\wwwhh\OneDrive\Desktop\fyp backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%run ../database.ipynb

In [3]:
image_mapping_list = get_all_image_mapping()

<h2>RESNET152<h2>

In [14]:
model = models.resnet152(pretrained=True)
model.eval()
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

def tensor_to_bytes(tensor):
    buffer = io.BytesIO()
    torch.save(tensor, buffer)
    return buffer.getvalue()

# Function to extract feature vector using ResNet152
def extract_features(image):
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image_tensor = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        features = feature_extractor(image_tensor).squeeze()
    return features, image_tensor.squeeze(0)

def calculate_similarity(chinese_image, sumerian_image):
    chinese_features, chinese_tensor = extract_features(chinese_image)
    sumerian_features, sumerian_tensor = extract_features(sumerian_image)
    similarity = 1 - cosine(chinese_features.numpy(), sumerian_features.numpy())
    ssim_similarity = float(ssim(chinese_tensor.numpy(), sumerian_tensor.numpy(), data_range=1.0, channel_axis=0))
    return chinese_features, sumerian_features, similarity, ssim_similarity

In [15]:
for i in range(len(image_mapping_list)):
    try:
        chinese_character = Image.open(io.BytesIO(image_mapping_list[i]["chinese_image"]))
        sumerian_character = Image.open(io.BytesIO(image_mapping_list[i]["sumerian_image"]))
        
        chinese_image_features, sumerian_image_features, similarity_score, ssim_similarity  = calculate_similarity(chinese_character, sumerian_character)
        
        image_mapping_list[i]["resnet_similarity_score"] = similarity_score
        image_mapping_list[i]["resnet_chinese_image_features"] = tensor_to_bytes(chinese_image_features)
        image_mapping_list[i]["resnet_sumerian_image_features"] = tensor_to_bytes(sumerian_image_features)
        image_mapping_list[i]["resnet_ssim_similarity_score"] = ssim_similarity
        
        update_image_mapping(image_mapping_list[i])
        
        # Print out the similarity score with index and images for better traceability
        print(f"Similarity Score for pair {i}: {similarity_score:.4f}")
    
    except Exception as e:
        print(f"Error processing pair {i}: {e}")

Going to update
Updated document with ID: 1d18dbd4-d902-4798-b261-b9f81cea8e9f
Similarity Score for pair 0: 0.8720
Going to update
Updated document with ID: d1910018-a4de-470b-9a43-9369ab50722d
Similarity Score for pair 1: 0.9429
Going to update
Updated document with ID: 4500f56a-82dd-42e8-ad67-ba731c11d5ed
Similarity Score for pair 2: 0.9460
Going to update
Updated document with ID: 52537fe9-e260-439b-83e0-3e5bd6de1423
Similarity Score for pair 3: 0.7159
Going to update
Updated document with ID: c41bd716-06e2-4a78-9392-f9d2c4de5da3
Similarity Score for pair 4: 0.7403
Going to update
Updated document with ID: be2ecd7a-2db5-4c62-bd47-9f4d7671d0ef
Similarity Score for pair 5: 0.8172
Going to update
Updated document with ID: a39c14e0-54d9-4a31-a665-b560113371a7
Similarity Score for pair 6: 0.7259
Going to update
Updated document with ID: 1ba492a2-521f-4c27-a240-e068815b6473
Similarity Score for pair 7: 0.8654
Going to update
Updated document with ID: e57d13f7-2daa-4bdf-9de1-b7ae26e684d0
S

KeyboardInterrupt: 

<h2>DINOV2<h2>

In [ ]:
def array_to_bytes(array):
    buffer = io.BytesIO()
    np.save(buffer, array)
    return buffer.getvalue()

model = AutoModel.from_pretrained('facebook/dinov2-base')
model.eval()


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


for i in range(len(image_mapping_list)):

    chinese_character = Image.open(io.BytesIO(image_mapping_list[i]["chinese_image"]))
    sumerian_character = Image.open(io.BytesIO(image_mapping_list[i]["sumerian_image"]))

    if chinese_character.mode == 'RGBA':
        chinese_character = chinese_character.convert('RGB')
    if sumerian_character.mode == 'RGBA':
        sumerian_character = sumerian_character.convert('RGB')

    chinese_tensor = preprocess(chinese_character).unsqueeze(0)
    sumerian_tensor = preprocess(sumerian_character).unsqueeze(0)

    with torch.no_grad():
        chinese_outputs = model(chinese_tensor).last_hidden_state
        sumerian_outputs = model(sumerian_tensor).last_hidden_state
    
    chinese_features = chinese_outputs.mean(dim=1).squeeze().numpy()
    sumerian_features = sumerian_outputs.mean(dim=1).squeeze().numpy()
    chinese_features_bytes = array_to_bytes(chinese_tensor)
    sumerian_features_bytes = array_to_bytes(sumerian_tensor)


    similarity = 1 - cosine(chinese_features, sumerian_features)
    ssim_similarity = float(ssim(chinese_tensor.squeeze(0).numpy(), sumerian_tensor.squeeze(0).numpy(), data_range=1.0, channel_axis=0))
    
    image_mapping_list[i]["dinov2_similarity_score"] = similarity
    image_mapping_list[i]["dinov2_ssim_similarity_score"] = ssim_similarity
    image_mapping_list[i]["dino_v2_chinese_image_features"] = chinese_features_bytes
    image_mapping_list[i]["dino_v2_sumerian_image_features"] = sumerian_features_bytes
    update_image_mapping(image_mapping_list[i])

    print(f"Similarity between the two images - Cosine: {similarity}, SSIM: {ssim_similarity}")

Going to update
Updated document with ID: 1d18dbd4-d902-4798-b261-b9f81cea8e9f
Similarity between the two images - Cosine: 0.660967635642463, SSIM: 0.5176234841346741
Going to update
Updated document with ID: d1910018-a4de-470b-9a43-9369ab50722d
Similarity between the two images - Cosine: 0.7946987734560081, SSIM: 0.580201268196106
Going to update
Updated document with ID: 4500f56a-82dd-42e8-ad67-ba731c11d5ed
Similarity between the two images - Cosine: 0.8111987889172739, SSIM: 0.5639266967773438
Going to update
Updated document with ID: 52537fe9-e260-439b-83e0-3e5bd6de1423
Similarity between the two images - Cosine: 0.5303084938551546, SSIM: 0.4474916458129883
Going to update
Updated document with ID: c41bd716-06e2-4a78-9392-f9d2c4de5da3
Similarity between the two images - Cosine: 0.6465225159001617, SSIM: 0.4867890775203705
Going to update
Updated document with ID: be2ecd7a-2db5-4c62-bd47-9f4d7671d0ef
Similarity between the two images - Cosine: 0.7760820128265712, SSIM: 0.45184728503

<h1>SSIM<h1>

In [22]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

for i in range(len(image_mapping_list)):
    chinese_character = Image.open(io.BytesIO(image_mapping_list[i]["chinese_image"]))
    sumerian_character = Image.open(io.BytesIO(image_mapping_list[i]["sumerian_image"]))

    if chinese_character.mode == 'RGBA':
        chinese_character = chinese_character.convert('RGB')
    if sumerian_character.mode == 'RGBA':
        sumerian_character = sumerian_character.convert('RGB')

    chinese_tensor = preprocess(chinese_character).unsqueeze(0)
    sumerian_tensor = preprocess(sumerian_character).unsqueeze(0)

    ssim_similarity = float(ssim(chinese_tensor.squeeze(0).numpy(), sumerian_tensor.squeeze(0).numpy(), data_range=1.0, channel_axis=0))
    
    image_mapping_list[i]["ssim_similarity_score"] = ssim_similarity
    update_image_mapping(image_mapping_list[i])

Going to update
Updated document with ID: 1d18dbd4-d902-4798-b261-b9f81cea8e9f
Going to update
Updated document with ID: d1910018-a4de-470b-9a43-9369ab50722d
Going to update
Updated document with ID: 4500f56a-82dd-42e8-ad67-ba731c11d5ed
Going to update
Updated document with ID: 52537fe9-e260-439b-83e0-3e5bd6de1423
Going to update
Updated document with ID: c41bd716-06e2-4a78-9392-f9d2c4de5da3
Going to update
Updated document with ID: be2ecd7a-2db5-4c62-bd47-9f4d7671d0ef
Going to update
Updated document with ID: a39c14e0-54d9-4a31-a665-b560113371a7
Going to update
Updated document with ID: 1ba492a2-521f-4c27-a240-e068815b6473
Going to update
Updated document with ID: e57d13f7-2daa-4bdf-9de1-b7ae26e684d0
Going to update
Updated document with ID: 95590602-1cc8-467a-af5d-3da0d10a5272
Going to update
Updated document with ID: 99048e1e-3bef-4268-a943-5aa6728cab65
Going to update
Updated document with ID: 7e7f33cd-5ee8-430c-adbc-ec2e0f448fc7
Going to update
Updated document with ID: 3dcb22d4-4

In [ ]:
image_mapping_list = get_all_image_mapping()
import pandas as pd
df = pd.DataFrame(image_mapping_list)

In [8]:
# Add rank columns (higher similarity = lower rank number)
df['resnet152_rank'] = df['resnet_similarity_score'].rank(ascending=False, method='min')
df['dinov2_rank'] = df['dinov2_similarity_score'].rank(ascending=False, method='min')
df['ssim_rank'] = df['ssim_similarity_score'].rank(ascending=False, method='min')

# View top-ranked pairs for each method
print("Top 5 by ResNet152:")
print(df.sort_values('resnet_similarity_score', ascending=False)[['number', 'resnet_similarity_score']].head(5))
print("\nTop 5 by DINOv2:")
print(df.sort_values('dinov2_similarity_score', ascending=False)[['number', 'dinov2_similarity_score']].head(5))
print("\nTop 5 by SSIM:")
print(df.sort_values('ssim_similarity_score', ascending=False)[['number', 'ssim_similarity_score']].head(5))

Top 5 by ResNet152:
     number  resnet_similarity_score
102      20                 0.975444
123      25                 0.953893
2         1                 0.946002
1         1                 0.942875
533      96                 0.940092

Top 5 by DINOv2:
     number  dinov2_similarity_score
102      20                 0.914948
343      67                 0.894203
123      25                 0.885039
196      35                 0.865341
80       15                 0.855002

Top 5 by SSIM:
     number  ssim_similarity_score
2         1               0.755177
122      25               0.754053
102      20               0.727812
1         1               0.721759
0         1               0.719214


In [10]:
# Pearson correlation (between similarity scores)
pearson_resnet_dinov2 = df['resnet_similarity_score'].corr(df['dinov2_similarity_score'])
pearson_dinov2_ssim = df['dinov2_similarity_score'].corr(df['ssim_similarity_score'])
pearson_resnet_ssim = df['resnet_similarity_score'].corr(df['ssim_similarity_score'])

print("Pearson Correlations:")
print(f"ResNet152 vs DINOv2: {pearson_resnet_dinov2:.3f}")
print(f"DINOv2 vs SSIM: {pearson_dinov2_ssim:.3f}")
print(f"ResNet152 vs SSIM: {pearson_resnet_ssim:.3f}")

# Spearman correlation (between similarity scores, based on ranks)
spearman_resnet_dinov2 = df['resnet_similarity_score'].corr(df['dinov2_similarity_score'], method='spearman')
spearman_dinov2_ssim = df['dinov2_similarity_score'].corr(df['ssim_similarity_score'], method='spearman')
spearman_resnet_ssim = df['resnet_similarity_score'].corr(df['ssim_similarity_score'], method='spearman')

print("\nSpearman Correlations:")
print(f"ResNet152 vs DINOv2: {spearman_resnet_dinov2:.3f}")
print(f"DINOv2 vs SSIM: {spearman_dinov2_ssim:.3f}")
print(f"ResNet152 vs SSIM: {spearman_resnet_ssim:.3f}")

Pearson Correlations:
ResNet152 vs DINOv2: 0.636
DINOv2 vs SSIM: 0.183
ResNet152 vs SSIM: 0.149

Spearman Correlations:
ResNet152 vs DINOv2: 0.631
DINOv2 vs SSIM: 0.152
ResNet152 vs SSIM: 0.143


In [12]:
stats = df[['resnet_similarity_score', 'dinov2_similarity_score', 'ssim_similarity_score']].agg(['mean', 'std']).T
stats['threshold_1sd'] = stats['mean'] + stats['std']  # 1 SD above mean

print("Statistics and Thresholds:")
print(stats)

# Extract thresholds
resnet152_threshold = stats.loc['resnet_similarity_score', 'threshold_1sd']
dinov2_threshold = stats.loc['dinov2_similarity_score', 'threshold_1sd']
ssim_threshold = stats.loc['ssim_similarity_score', 'threshold_1sd']

Statistics and Thresholds:
                             mean       std  threshold_1sd
resnet_similarity_score  0.755586  0.083659       0.839245
dinov2_similarity_score  0.634683  0.097350       0.732033
ssim_similarity_score    0.468604  0.099606       0.568210


In [ ]:
def tensor_to_bytes(tensor):
    buffer = io.BytesIO()
    torch.save(tensor, buffer)
    return buffer.getvalue()

In [21]:
for i in range(len(image_mapping_list)):
    cur_image_mapping = image_mapping_list[i]
    chinese_character = Image.open(io.BytesIO(image_mapping_list[i]["chinese_image"]))
    sumerian_character = Image.open(io.BytesIO(image_mapping_list[i]["sumerian_image"]))
    
    chinese_image_features = extract_features(chinese_character)
    sumerian_image_features = extract_features(sumerian_character)
    
    cur_image_mapping["resnet_chinese_image_features"] = tensor_to_bytes(chinese_image_features)
    cur_image_mapping["resnet_sumerian_image_features"] = tensor_to_bytes(sumerian_image_features)
    
    update_image_mapping(cur_image_mapping)

Going to update
Updated document with ID: 1d18dbd4-d902-4798-b261-b9f81cea8e9f
Going to update
Updated document with ID: d1910018-a4de-470b-9a43-9369ab50722d
Going to update
Updated document with ID: 4500f56a-82dd-42e8-ad67-ba731c11d5ed
Going to update
Updated document with ID: 52537fe9-e260-439b-83e0-3e5bd6de1423
Going to update
Updated document with ID: c41bd716-06e2-4a78-9392-f9d2c4de5da3
Going to update
Updated document with ID: be2ecd7a-2db5-4c62-bd47-9f4d7671d0ef
Going to update
Updated document with ID: a39c14e0-54d9-4a31-a665-b560113371a7
Going to update
Updated document with ID: 1ba492a2-521f-4c27-a240-e068815b6473
Going to update
Updated document with ID: e57d13f7-2daa-4bdf-9de1-b7ae26e684d0
Going to update
Updated document with ID: 95590602-1cc8-467a-af5d-3da0d10a5272
Going to update
Updated document with ID: 99048e1e-3bef-4268-a943-5aa6728cab65
Going to update
Updated document with ID: 7e7f33cd-5ee8-430c-adbc-ec2e0f448fc7
Going to update
Updated document with ID: 3dcb22d4-4

In [23]:
import torch
from transformers import AutoModel
from torchvision import transforms
from PIL import Image
import io
import numpy as np
from scipy.spatial.distance import cosine

# Load the DINOv2 model
model = AutoModel.from_pretrained('facebook/dinov2-base')
model.eval()  # Set to evaluation mode

# Define preprocessing (matching your ResNet152 setup)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to convert NumPy array to bytes
def array_to_bytes(array):
    buffer = io.BytesIO()
    np.save(buffer, array)  # Serialize NumPy array to bytes
    return buffer.getvalue()

# Process each image pair in the list
for i in range(len(image_mapping_list)):
    # Load images from bytes
    chinese_character = Image.open(io.BytesIO(image_mapping_list[i]["chinese_image"]))
    sumerian_character = Image.open(io.BytesIO(image_mapping_list[i]["sumerian_image"]))

    # Handle RGBA conversion if needed
    if chinese_character.mode == 'RGBA':
        chinese_character = chinese_character.convert('RGB')
    if sumerian_character.mode == 'RGBA':
        sumerian_character = sumerian_character.convert('RGB')

    # Preprocess images
    chinese_tensor = preprocess(chinese_character).unsqueeze(0)  # Add batch dimension
    sumerian_tensor = preprocess(sumerian_character).unsqueeze(0)

    # Extract features with DINOv2
    with torch.no_grad():
        chinese_outputs = model(chinese_tensor).last_hidden_state
        sumerian_outputs = model(sumerian_tensor).last_hidden_state

    # Average the features across the sequence dimension and convert to NumPy
    chinese_features = chinese_outputs.mean(dim=1).squeeze().numpy()  # Shape: (768,)
    sumerian_features = sumerian_outputs.mean(dim=1).squeeze().numpy()  # Shape: (768,)

    # Convert features to bytes for database storage
    chinese_features_bytes = array_to_bytes(chinese_features)
    sumerian_features_bytes = array_to_bytes(sumerian_features)

    # Calculate cosine similarity
    similarity = 1 - cosine(chinese_features, sumerian_features)

    # Update the mapping with features (as bytes) and similarity score
    image_mapping_list[i]["dino_v2_chinese_image_features"] = chinese_features_bytes
    image_mapping_list[i]["dino_v2_sumerian_image_features"] = sumerian_features_bytes

    # Save to database (assuming update_image_mapping handles bytes)
    update_image_mapping(image_mapping_list[i])

    print(f"Cosine similarity between the two images: {similarity}")

Going to update
Updated document with ID: 1d18dbd4-d902-4798-b261-b9f81cea8e9f
Cosine similarity between the two images: 0.660967635642463
Going to update
Updated document with ID: d1910018-a4de-470b-9a43-9369ab50722d
Cosine similarity between the two images: 0.7946987734560081
Going to update
Updated document with ID: 4500f56a-82dd-42e8-ad67-ba731c11d5ed
Cosine similarity between the two images: 0.8111987889172739
Going to update
Updated document with ID: 52537fe9-e260-439b-83e0-3e5bd6de1423
Cosine similarity between the two images: 0.5303084938551546
Going to update
Updated document with ID: c41bd716-06e2-4a78-9392-f9d2c4de5da3
Cosine similarity between the two images: 0.6465225159001617
Going to update
Updated document with ID: be2ecd7a-2db5-4c62-bd47-9f4d7671d0ef
Cosine similarity between the two images: 0.7760820128265712
Going to update
Updated document with ID: a39c14e0-54d9-4a31-a665-b560113371a7
Cosine similarity between the two images: 0.6793851999192143
Going to update
Updat

In [6]:
from pymongo import MongoClient

# Connect to MongoDB (adjust URI as needed)
client = MongoClient("mongodb://localhost:27017/")  # Default local MongoDB
db = client["final-year-project"]  # Replace with your DB name
collection = db["image_mapping"]  # Replace with your collection name

# Function to delete a field from a document
def delete_field(doc_id, field_name):
    collection.update_one(
        {"_id": doc_id},  # Match document by _id
        {"$unset": {field_name: ""}}  # Remove the specified field
    )

# Example: Delete "resnet_ssim_similarity_score" for all documents in image_mapping_list
for i in range(len(image_mapping_list)):
    doc_id = image_mapping_list[i]["_id"]  # Assumes _id exists
    field_to_delete = "dinov2_ssim_similarity_score"
    
    try:
        delete_field(doc_id, field_to_delete)
        print(f"Deleted field '{field_to_delete}' from document {doc_id}")
    except Exception as e:
        print(f"Error deleting field for document {doc_id}: {e}")

# Alternative: Delete field from all documents in the collection
def delete_field_from_all(field_name):
    result = collection.update_many(
        {},  # Match all documents
        {"$unset": {field_name: ""}}
    )
    print(f"Deleted '{field_name}' from {result.modified_count} documents")

# Run for all documents
delete_field_from_all("resnet_ssim_similarity_score")

Deleted field 'dinov2_ssim_similarity_score' from document 1d18dbd4-d902-4798-b261-b9f81cea8e9f
Deleted field 'dinov2_ssim_similarity_score' from document d1910018-a4de-470b-9a43-9369ab50722d
Deleted field 'dinov2_ssim_similarity_score' from document 4500f56a-82dd-42e8-ad67-ba731c11d5ed
Deleted field 'dinov2_ssim_similarity_score' from document 52537fe9-e260-439b-83e0-3e5bd6de1423
Deleted field 'dinov2_ssim_similarity_score' from document c41bd716-06e2-4a78-9392-f9d2c4de5da3
Deleted field 'dinov2_ssim_similarity_score' from document be2ecd7a-2db5-4c62-bd47-9f4d7671d0ef
Deleted field 'dinov2_ssim_similarity_score' from document a39c14e0-54d9-4a31-a665-b560113371a7
Deleted field 'dinov2_ssim_similarity_score' from document 1ba492a2-521f-4c27-a240-e068815b6473
Deleted field 'dinov2_ssim_similarity_score' from document e57d13f7-2daa-4bdf-9de1-b7ae26e684d0
Deleted field 'dinov2_ssim_similarity_score' from document 95590602-1cc8-467a-af5d-3da0d10a5272
Deleted field 'dinov2_ssim_similarity_sc